In [ ]:
#podział na odchylenia wszystkie i odchylenia złe, bez odchyleń

#pca
#miernik na podstawie ostatniej wartości
#użycie wag do LDA w czasie

#granica LDA

#50 pacjentów zmarłych i 50 zywych

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import plotly.express as px
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
import random
import plotly.graph_objs as go

### dane

In [3]:
df_prep = pd.read_csv("..\chapter3\gasometry_prepared_data.csv")
cols=["BETET",	"CO2TET",	"HCO3ACTE",	"HCO3STTE",	"O2SATTET",	"O2TET", "IONH"]
my_blue="#0064B2";
my_red="#D61600";

In [239]:
df_miernik=df_prep.copy()
vars1=["BETET", "O2TET"]
vars2=["IONH", "CO2TET"]

#wyliczenie wartości poza normą świadczące o zakwaszeniu organizmu
df_miernik[["BETET_kw", "O2TET_kw"]]=df_miernik[vars1].applymap(lambda x: 0 if x >= 0 else abs(x))
df_miernik[["IONH_kw", "CO2TET_kw"]]=df_miernik[vars2].applymap(lambda x: 0 if x <= 1 else x-1)

df_prep=df_miernik

C:\Users\dorot\AppData\Local\Temp\ipykernel_43564\991917892.py:6: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\dorot\AppData\Local\Temp\ipykernel_43564\991917892.py:7: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [240]:
random.seed(42)  
df_zgon = df_prep.groupby('PACJENT_NR').last()
sample_zgon = random.sample(list(df_zgon[df_zgon['ZGON'] ==1 ].index), 40)
sample_nzgon = random.sample(list(df_zgon[df_zgon['ZGON'] == 0].index), 40)
indexes=[*sample_zgon, *sample_nzgon]
pacjenci_80 = df_zgon.loc[df_zgon.index.isin(indexes)].index
data80=df_prep.loc[df_prep["PACJENT_NR"].isin(pacjenci_80)].reset_index(drop=True)
data27=df_prep.loc[~df_prep["PACJENT_NR"].isin(pacjenci_80)].reset_index(drop=True)

In [241]:
data80

,Unnamed: 0,PACJENT_NR,BADANIE_NR,BETET,CO2TET,HCO3ACTE,HCO3STTE,O2SATTET,O2TET,IONH,ZGON,BETET_kw,O2TET_kw,IONH_kw,CO2TET_kw
0,60,3,1,0.478261,1.42,0.883333,0.750,0.74,2.148,1.008167,0.0,0.000000,0.00,0.008167,0.42
1,61,3,2,1.782609,2.06,1.700000,1.675,0.90,3.924,0.795364,0.0,0.000000,0.00,0.000000,1.06
2,62,3,3,1.804348,1.24,1.716667,2.050,0.92,3.984,0.147539,0.0,0.000000,0.00,0.000000,0.24
3,63,3,4,2.826087,2.18,2.383333,2.600,0.56,0.936,0.381504,0.0,0.000000,0.00,0.000000,1.18
4,64,3,5,2.652174,2.31,2.633333,2.875,0.76,1.708,0.301894,0.0,0.000000,0.00,0.000000,1.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,2929,145,5,-0.717391,1.63,0.133333,-0.350,0.18,0.420,2.294287,1.0,0.717391,0.00,1.294287,0.63
2166,2930,145,6,-0.891304,1.39,-0.016667,-0.450,0.76,1.640,2.254399,1.0,0.891304,0.00,1.254399,0.39
2167,2931,145,7,-1.065217,1.71,-0.050000,-0.650,-0.06,0.316,2.708690,1.0,1.065217,0.00,1.708690,0.71
2168,2932,145,8,-0.695652,1.15,0.066667,-0.275,0.62,1.600,1.857967,1.0,0.695652,0.00,0.857967,0.15


In [243]:
cols=data80.columns[3:-1]
cols=["BETET_kw", "O2TET_kw", "IONH_kw", "CO2TET_kw"]

### PCA

In [244]:
X_data=data80[cols].reset_index(drop=True)

pca = PCA()
components = pca.fit_transform(X_data)

pca.explained_variance_ratio_


array([0.71926015, 0.20923469, 0.05056287, 0.02094229])

In [246]:
# Wyjaśniona wariancja
explained_variance_ratio = pca.explained_variance_ratio_

# Kumulatywna suma wyjaśnionej wariancji
cumulative_explained_variance_ratio = explained_variance_ratio.cumsum()

# Tworzenie osypiska (scree plot) za pomocą Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(explained_variance_ratio) + 1)), y=explained_variance_ratio,
                         mode='lines+markers', name='Explained Variance Ratio'))
fig.add_trace(go.Scatter(x=list(range(1, len(cumulative_explained_variance_ratio) + 1)), 
                         y=cumulative_explained_variance_ratio,
                         mode='lines+markers', name='Cumulative Explained Variance Ratio'))

# Dodanie pionowej linii dla 95% wyjaśnionej wariancji
fig.add_shape(type="line",
              x0=1, y0=0.95, x1=len(explained_variance_ratio), y1=0.95,
              line=dict(color="red", width=2, dash="dash"))

# Dodanie tekstu dla 95% wyjaśnionej wariancji
fig.add_annotation(
            x=len(explained_variance_ratio),
            y=0.95,
            text="95% Explained Variance",
            showarrow=True,
            arrowhead=1,
            arrowcolor="red",
            arrowsize=1,
            arrowwidth=2,
            ax=50,
            ay=-50,
            font=dict(color="red", size=12),
            align="center"
        )

# Konfiguracja osi i tytułu
fig.update_layout(title='Scree Plot - Explained Variance Ratio',
                  xaxis_title='Principal Component',
                  yaxis_title='Explained Variance Ratio')

# Wyświetlenie wykresu
fig.show()


In [248]:
X_data=data80[cols].reset_index(drop=True)
y_data=data80['ZGON']

pca = PCA(n_components=3)
components = pca.fit_transform(X_data)
components_test=pca.transform(data27[cols])

pca.explained_variance_ratio_

array([0.71926015, 0.20923469, 0.05056287])

In [249]:
# Wagi (współczynniki) PCA
weights_pca = pca.components_
weights_df = pd.DataFrame(weights_pca, columns=X_data.columns)
weights_df 

,BETET_kw,O2TET_kw,IONH_kw,CO2TET_kw
0,0.115341,0.006166,0.546195,0.829656
1,0.472885,-0.060875,0.702797,-0.527968
2,0.132447,0.990948,-0.015154,-0.015801


In [250]:
transformed_df = pd.DataFrame(components, columns=[f'PC{i+1}' for i in range(components.shape[1])])
transformed_df[['PACJENT_NR', 'BADANIE_NR', 'ZGON']] = data80[['PACJENT_NR', 'BADANIE_NR', 'ZGON']]
transformed_df

,PC1,PC2,PC3,PACJENT_NR,BADANIE_NR,ZGON
0,-0.519466,-0.054108,-0.135803,3,1,0.0
1,0.007053,-0.397747,-0.145792,3,2,0.0
2,-0.673265,0.035186,-0.132835,3,3,0.0
3,0.106612,-0.461104,-0.147688,3,4,0.0
4,0.214468,-0.529739,-0.149742,3,5,0.0
...,...,...,...,...,...,...
2165,0.439979,1.078144,-0.063595,145,5,1.0
2166,0.239134,1.259064,-0.036164,145,6,1.0
2167,0.772815,1.491629,-0.025071,145,7,1.0
2168,-0.199080,1.014643,-0.052278,145,8,1.0


In [153]:
pc_comp=columns=[f'PC{i+1}' for i in range(components.shape[1])]
pc_comp

['PC1', 'PC2', 'PC3']

In [251]:
transformed_test_df = pd.DataFrame(components_test, columns=pc_comp)
transformed_test_df[['PACJENT_NR', 'BADANIE_NR', 'ZGON']] = data27[['PACJENT_NR', 'BADANIE_NR', 'ZGON']]
transformed_test_df

,PC1,PC2,PC3,PACJENT_NR,BADANIE_NR,ZGON
0,0.618066,0.099585,0.551944,1,1,0.0
1,3.883445,-0.451713,-0.230533,1,2,0.0
2,1.526227,-0.102323,-0.180434,1,3,0.0
3,1.061641,-0.141688,-0.170071,1,4,0.0
4,0.993261,-0.006304,-0.169184,1,5,0.0
...,...,...,...,...,...,...
759,-0.329456,-0.217319,0.447328,140,20,1.0
760,-0.183767,-0.276315,-0.142157,140,21,1.0
761,-0.539780,-0.056594,-0.016449,140,22,1.0
762,-0.462916,-0.125782,0.334906,140,23,1.0


### LDA z ostatnich wartości

In [252]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix

In [253]:
X_train_data_lda=transformed_df.groupby("PACJENT_NR").last().reset_index()
X_test_data_lda=transformed_test_df.groupby("PACJENT_NR").last().reset_index()

In [254]:
X_train_data_lda

,PACJENT_NR,PC1,PC2,PC3,BADANIE_NR,ZGON
0,3,-0.872382,0.161899,-0.129043,41,0.0
1,4,-0.747934,0.082703,-0.131413,34,0.0
2,6,-0.872382,0.161899,-0.129043,8,0.0
3,7,-0.586478,0.711215,0.281557,12,1.0
4,8,0.033709,-0.507655,1.471120,10,0.0
...,...,...,...,...,...,...
75,134,0.702278,-0.901219,0.903390,35,1.0
76,139,0.356076,-0.625095,-0.061261,32,0.0
77,142,1.052207,-0.999441,-0.165983,21,0.0
78,144,1.143683,-1.121064,-0.167438,11,0.0


In [255]:
# Definiowanie zmiennych niezależnych i celu
X_train = X_train_data_lda.drop(columns=["ZGON", "BADANIE_NR", "PACJENT_NR"])
y_train = X_train_data_lda['ZGON']

X_test = X_test_data_lda.drop(columns=["ZGON", "BADANIE_NR", "PACJENT_NR"])
y_test = X_test_data_lda['ZGON']

# Tworzenie i trenowanie modelu LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predykcje na zbiorze testowym
y_pred = lda.predict(X_test)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.5925925925925926
Confusion Matrix:
[[10  2]
 [ 9  6]]


In [256]:
lda_weights = lda.coef_
print(lda_weights)
print(lda.intercept_)

[[0.52929258 0.42146241 0.71145023]]
[-0.57297069]


In [213]:
train_data_after_lda= lda.transform(transformed_df[pc_comp])
X_lda_with_zgon = pd.DataFrame(np.column_stack((train_data_after_lda, transformed_df[['ZGON', "BADANIE_NR", "PACJENT_NR"]])), columns=["LDA", "ZGON", "BADANIE_NR", "PACJENT_NR"])
X_lda_with_zgon

,LDA,ZGON,BADANIE_NR,PACJENT_NR
0,-0.520343,0.0,1.0,3.0
1,-0.989835,0.0,2.0,3.0
2,-1.370449,0.0,3.0,3.0
3,-0.985555,0.0,4.0,3.0
4,-1.115386,0.0,5.0,3.0
...,...,...,...,...
2165,0.430300,1.0,5.0,145.0
2166,0.195590,1.0,6.0,145.0
2167,0.685028,1.0,7.0,145.0
2168,0.028346,1.0,8.0,145.0


In [258]:
test_data_after_lda= lda.transform(transformed_test_df[pc_comp])
X_lda_with_zgon_test = pd.DataFrame(np.column_stack((test_data_after_lda, transformed_test_df[['ZGON', "BADANIE_NR", "PACJENT_NR"]])), columns=["LDA", "ZGON", "BADANIE_NR", "PACJENT_NR"])
X_lda_with_zgon_test

,LDA,ZGON,BADANIE_NR,PACJENT_NR
0,0.148926,0.0,1.0,1.0
1,0.889771,0.0,2.0,1.0
2,0.049969,0.0,3.0,1.0
3,-0.151249,0.0,4.0,1.0
4,-0.134295,0.0,5.0,1.0
...,...,...,...,...
759,-0.410680,1.0,20.0,140.0
760,-0.700254,1.0,21.0,140.0
761,-0.705298,1.0,22.0,140.0
762,-0.499051,1.0,23.0,140.0


In [259]:
decision_boundary = lda.coef_[0]
decision_boundary

array([0.52929258, 0.42146241, 0.71145023])

In [260]:
X_test_data_lda

,PACJENT_NR,PC1,PC2,PC3,BADANIE_NR,ZGON
0,1,-0.051022,-0.360790,-0.144685,50,0.0
1,2,-0.872382,0.161899,-0.129043,10,0.0
2,29,-0.872358,0.161655,-0.125079,32,0.0
3,32,1.091690,1.352484,0.549165,31,1.0
4,37,2.216406,-1.826492,0.208558,51,1.0
5,46,-0.872382,0.161899,-0.129043,40,0.0
6,53,3.391138,-1.239261,0.853896,20,1.0
7,61,-0.867367,0.182459,-0.123284,34,1.0
8,62,3.089422,-0.771376,-0.211679,11,1.0
9,67,-0.864638,0.085440,1.115588,37,0.0


In [261]:
X_test_data_lda["ZGON"]

0     0.0
1     0.0
2     0.0
3     1.0
4     1.0
5     0.0
6     1.0
7     1.0
8     1.0
9     0.0
10    0.0
11    0.0
12    1.0
13    0.0
14    1.0
15    1.0
16    1.0
17    1.0
18    0.0
19    0.0
20    1.0
21    1.0
22    0.0
23    1.0
24    1.0
25    0.0
26    1.0
Name: ZGON, dtype: float64

In [262]:
log_probabilities = lda.predict_proba(X_test_data_lda.drop(columns=["ZGON", "BADANIE_NR", "PACJENT_NR"]))
log_probabilities 

array([[0.70160916, 0.29839084],
       [0.74236352, 0.25763648],
       [0.74184095, 0.25815905],
       [0.27576297, 0.72423703],
       [0.50531554, 0.49468446],
       [0.74236352, 0.25763648],
       [0.21296654, 0.78703346],
       [0.73940395, 0.26059605],
       [0.35743086, 0.64256914],
       [0.55006222, 0.44993778],
       [0.74236352, 0.25763648],
       [0.74236352, 0.25763648],
       [0.60161941, 0.39838059],
       [0.48833572, 0.51166428],
       [0.74078923, 0.25921077],
       [0.23381091, 0.76618909],
       [0.0976293 , 0.9023707 ],
       [0.69685826, 0.30314174],
       [0.66271454, 0.33728546],
       [0.73841609, 0.26158391],
       [0.62532103, 0.37467897],
       [0.74236352, 0.25763648],
       [0.48742653, 0.51257347],
       [0.12319041, 0.87680959],
       [0.66543259, 0.33456741],
       [0.70060183, 0.29939817],
       [0.57999227, 0.42000773]])

In [263]:
np.log(12/15)

-0.2231435513142097

In [266]:
# Podziel dane na dwie ramki danych
df_zgon_0 = X_lda_with_zgon[X_lda_with_zgon['ZGON'] == 0]
df_zgon_1 = X_lda_with_zgon[X_lda_with_zgon['ZGON'] == 1]

# fig0 = make_subplots(rows=1, cols=2, subplot_titles=['Zgon: nie', 'Zgon: tak'])
# Wykres dla ZGON=0
fig1 = px.line(df_zgon_0, x='BADANIE_NR', y='LDA', color='PACJENT_NR', 
              labels={'BADANIE_NR': 'BADANIE_NR', 'LDA': 'LDA', 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig1.update_xaxes(title_text='Numer badania')
fig1.update_yaxes(title_text='LDA')
fig1.update_layout(showlegend=False)

# Wykres dla ZGON=1
fig2 = px.line(df_zgon_1, x='BADANIE_NR', y='LDA', color='PACJENT_NR', 
               labels={'BADANIE_NR': 'BADANIE_NR', 'LDA': 'LDA', 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig2.update_xaxes(title_text='Numer badania')
fig2.update_yaxes(title_text='LDA zmarli')
fig2.update_layout(showlegend=False)

fig1.update_yaxes(range=[-3, 5])
fig2.update_yaxes(range=[-3, 5])

fig1.update_xaxes(range=[0, 65])
fig2.update_xaxes(range=[0, 65])

fig1.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=20),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=20)
    )
)
fig2.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=24),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=24)
    )
)
fig1.update_layout(template="plotly_white")
fig2.update_layout(template="plotly_white")
# Wyświetl oba wykresy
fig1.show()
fig2.show()